## Chapter-20


### Descriptor example: attribute validation


In [1]:
class Quantity:
    
    def __init__(self, name):
        self.name = name
    
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.name] = value
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price):
        
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        
        return self.weight * self.price

In [2]:
l = LineItem('nuts', 100, 0)

ValueError: value must be > 0

In [3]:
LineItem.weight

In [20]:
# understand attr

class Cls:
    
    att = 1
    
    def __init__(self, att):
        self.att = att
        
    @property
    def att(self):
        return 3
    
c = Cls(2)
c.att

AttributeError: can't set attribute

In [22]:
# understand attr

class Cls:
    
    att = 1
    
    def __init__(self, att):
        self.att = att
        
#     @property
#     def att(self):
#         return 3
    
c = Cls(2)
print(c.att)
print(Cls.att)

2
1


In [23]:
# understand attr

class Cls:
    
    att = 1
    
#     def __init__(self, att):
#         self.att = att
        
    @property
    def att(self):
        return 3
    
c = Cls()
print(c.att)
print(Cls.att)

3


In [25]:
class Cls:
    
#     att = 1
    
    def __init__(self, att):
        self.att = att
        
    @property
    def att(self):
        return self.att
    
c = Cls(2)
print(c.att)
print(Cls.att)

AttributeError: can't set attribute

In [30]:
class Cls:
    
    att = 1
    
#     def __init__(self, att):
#         self.att = att
        
#     @property
    def att(self):
        return 3
    
c = Cls()
print(c.att())
print(Cls.att)

3
<function Cls.att at 0x7fa73456dea0>


In [32]:
class Cls:
    
    att = 1
    
    def __init__(self, att):
        self.att = att
        
c = Cls(2)

print(c.att)
print(Cls.att)

2
1


#### conclusion:

清楚以下概念：

- 什么是 class attribute 
- 什么是 class property
- 什么是 instance attribute
- class attribute 和 class property 什么时候表示相同的对象


In [41]:
class Cls:
    
    att = 1 # class attribute
    
    def __init__(self, att):
        self.att = att #  instance attribute
        
    @property
    def att(self):  # class property
        return 3
    
    @att.setter
    def att(self, value):
        return 4

In [42]:
c = Cls(2)

In [43]:
c.att

3

### automatic storage attribute names


In [44]:
class Quantity:
    
    _counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter 
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls._counter += 1
        
    def __get__(self, instance, owner):
        return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
            
        else:
            raise ValueError('must be > 0')
            


In [45]:
nuts2 = LineItem('nuts2', 100, 2)

In [47]:
nuts2.weight, nuts2.price

(100, 2)

In [51]:
Quantity._counter

0

### LineItem take #5: a new descriptor type

In [18]:
class AutoStorage:
    
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter 
        self.storage_name = '_{}#()'.format(prefix, index)
        cls.__counter += 1
    
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        

In [19]:
import abc

class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(self, instance, value):
        '''return validated value or raise valueerror'''
        
    

In [20]:
class Quantity(Validated):
    '''a number greater than zero'''
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
    

In [21]:
class NonBlank(Validated):
    '''a string with at least one none-space character'''
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
            
        return value
    

In [22]:
class LineItem:
    
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.weight = weight
        self.description = description
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    

In [14]:
l = LineItem('', 100, 1)

ValueError: value cannot be empty or blank

In [15]:
l2 = LineItem('cat', 100, 0)

ValueError: value must be > 0

In [23]:
l3 = LineItem('cat', 100, 1)

In [24]:
l3.weight, l3.price, l3.description

(1, 1, 'cat')

In [28]:
l3.weight.storage_name

AttributeError: 'int' object has no attribute 'storage_name'

In [2]:
### understand descriptor

class Descriptor:

    def __init__(self,initial_value=None, name='var'):
        self.name = name
        self.value = initial_value
        
    def __get__(self, obj, objtype):
        print('this is __get__({},{})'.format(obj, objtype))
        return self.value
    
    def __set__(self, obj, value):
        print('this is __set__({}{})'.format(obj, value))
        self.value = value
        
class Cls():
    
    des = Descriptor(initial_value='mike', name='desc')
    normal = 10


In [3]:
c = Cls()
c.des

this is __get__(<__main__.Cls object at 0x7f37dd343b38>,<class '__main__.Cls'>)


'mike'

In [4]:
c.des = 'partrick'

this is __set__(<__main__.Cls object at 0x7f37dd343b38>partrick)


In [5]:
c.normal

10

In [6]:
Cls.normal

10

In [ ]:
c.des

In [24]:
# re-analysis former example

class AutoStorage:
    
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter 
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    
    def __set__(self, instance, value):
        print('Auxx.__set__({},{})'.format(instance, value))
        setattr(instance, self.storage_name, value)
    
    def __get__(self, instance, owner):
        print('__get__({},{})'.format(instance, owner))
        if instance is None:
            return self
        else:
            print('getattr({},{})'.format(instance, self.storage_name))
            return getattr(instance, self.storage_name)
        
import abc

class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        print('Vaxx.__set__({},{})'.format(instance, value))
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(self, instance, value):
        '''return validated value or raise valueerror'''
        
class Quantity(Validated):
    '''a number greater than zero'''
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
    
class NonBlank(Validated):
    '''a string with at least one none-space character'''
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
            
        return value
    
    
class LineItem:
    
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        print('proceed self.weight')
        self.weight = weight
        print('proceed self.description')
        self.description = description
        print('proceed self.price')
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    

In [20]:
l = LineItem('cat', 100, 1)

proceed self.weight
Vaxx.__set__(<__main__.LineItem object at 0x7f37dca8c3c8>,100)
Auxx.__set__(<__main__.LineItem object at 0x7f37dca8c3c8>,100)
proceed self.description
Vaxx.__set__(<__main__.LineItem object at 0x7f37dca8c3c8>,cat)
Auxx.__set__(<__main__.LineItem object at 0x7f37dca8c3c8>,cat)
proceed self.price
Vaxx.__set__(<__main__.LineItem object at 0x7f37dca8c3c8>,1)
Auxx.__set__(<__main__.LineItem object at 0x7f37dca8c3c8>,1)


In [21]:
l.weight 

__get__(<__main__.LineItem object at 0x7f37dca8c3c8>,<class '__main__.LineItem'>)
getattr(<__main__.LineItem object at 0x7f37dca8c3c8>,_Quantity#0)


100

In [22]:
l.price

__get__(<__main__.LineItem object at 0x7f37dca8c3c8>,<class '__main__.LineItem'>)
getattr(<__main__.LineItem object at 0x7f37dca8c3c8>,_Quantity#1)


1

In [23]:
l.weight = 3

Vaxx.__set__(<__main__.LineItem object at 0x7f37dca8c3c8>,3)
Auxx.__set__(<__main__.LineItem object at 0x7f37dca8c3c8>,3)


In [25]:
l2 = LineItem('cat', 100, 1)

proceed self.weight
Auxx.__set__(<__main__.LineItem object at 0x7f37dc994e80>,100)
proceed self.description
Auxx.__set__(<__main__.LineItem object at 0x7f37dc994e80>,cat)
proceed self.price
Auxx.__set__(<__main__.LineItem object at 0x7f37dc994e80>,1)


In [26]:
l2.price = 3

Auxx.__set__(<__main__.LineItem object at 0x7f37dc994e80>,3)


### Overriding versus non-overriding descriptors


In [27]:
type(None)

NoneType

In [51]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split()[-1]


def display(obj):
    
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    
    elif cls in [type(None), int]:
        return repr(obj)
    
    else:
        return '<{} object>'.format(cls_name(obj))
    
def print_args(name, *args):
    pesudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pesudo_args))
    
class Overriding:
    '''a.k.a data descriptor or enforced descriptor'''
    def __get__(self, instance, owner):
        print('owner = {}'.format(owner))
        print_args('O_get', self, instance, owner)
        
    def __set__(self, instance, value):
        print_args('O_set', self, instance, value)
        
class OverridingNoGet:
    
    def __set__(self, instance, value):
        print('value = {}'.format(value))
        print_args('OG_set', self, instance, value)
        
class NonOverriding:
    
    def __get__(self,instance, owner):
        print('owner = {}'.format(owner))
        print_args('N_get', self, instance, owner)
        

class Managed:
    
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    
    
    def spam(self):
        
        print('-> Managed.spam({})'.format(display(self)))
        


In [31]:
class Class(object): pass

type(Class)

type

In [52]:
obj = Managed()

In [59]:
# obj.over

In [53]:
obj.over

owner = <class '__main__.Managed'>
-> Overriding.__O_get__(<Overriding object>, <Managed object>, <class Managed>)


In [54]:
Managed.over

owner = <class '__main__.Managed'>
-> Overriding.__O_get__(<Overriding object>, None, <class Managed>)


In [55]:
obj.over = 7

-> Overriding.__O_set__(<Overriding object>, <Managed object>, 7)


In [56]:
obj.over

owner = <class '__main__.Managed'>
-> Overriding.__O_get__(<Overriding object>, <Managed object>, <class Managed>)


In [57]:
obj.__dict__['over'] = 10

In [58]:
vars(obj)

{'over': 10}

In [60]:
# obj.over_no_get

obj.over_no_get

In [61]:
Managed.over_no_get


In [62]:
obj.over_no_get = 7

value = 7
-> OverridingNoGet.__OG_set__(<OverridingNoGet object>, <Managed object>, 7)


In [63]:
obj.over_no_get

In [64]:
obj.__dict__['over_no_get'] = 9

In [65]:
vars(obj)

{'over': 10, 'over_no_get': 9}

In [66]:
obj.over_no_get

9

In [67]:
# Non-overriding descriptor


In [68]:
obj = Managed()

obj.non_over

owner = <class '__main__.Managed'>
-> NonOverriding.__N_get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [70]:
obj.non_over = 7

In [71]:
obj.non_over

7

In [72]:
Managed.non_over

owner = <class '__main__.Managed'>
-> NonOverriding.__N_get__(<NonOverriding object>, None, <class Managed>)


In [73]:
del obj.non_over

In [74]:
obj.non_over

owner = <class '__main__.Managed'>
-> NonOverriding.__N_get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [76]:
# overwriting 

Managed.over = 1
Managed.over_no_get = 3
Managed.non_over = 5

obj.over, obj.over_no_get, obj.non_over

(1, 3, 5)

### Methods are descriptors


In [77]:
obj = Managed()
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x7f37dd339ef0>>

In [79]:
Managed.spam

<function __main__.Managed.spam>

In [80]:
obj.spam = 7

In [81]:
obj.spam

7

In [82]:
# 

import collections


class Text(collections.UserString):
    
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    
    def reverse(self):
        return self[::-1]


In [83]:
word = Text('forward')

In [84]:
word

Text('forward')

In [85]:
word.reverse()

Text('drawrof')

In [86]:
Text.reverse(Text('backword'))

Text('drowkcab')

In [87]:
type(Text.reverse)

function

In [88]:
type(word.reverse)

method

In [89]:
list(map(Text.reverse, ['rapaid', (10,20,30), Text('stressed')]))

['diapar', (30, 20, 10), Text('desserts')]

In [90]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

In [91]:
word.reverse

<bound method Text.reverse of Text('forward')>

In [92]:
word.reverse.__self__

Text('forward')

In [93]:
dir(word)

['__abstractmethods__',
 '__add__',
 '__class__',
 '__complex__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__float__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__int__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'capitalize',
 'casefold',
 'center',
 'count',
 'data',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower'

In [94]:
Text('backword')

Text('backword')